In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from copy import deepcopy

# Utility functions
def set_npseed(seed):
    np.random.seed(seed)

def set_torchseed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def compute_hyperplane_distance(v1, v2):
    """
    Compute distance between two hyperplanes.
    Uses minimum of ||v1_norm - v2_norm|| and ||v1_norm + v2_norm|| 
    to account for sign ambiguity.
    
    Args:
        v1: first hyperplane normal vector
        v2: second hyperplane normal vector
    
    Returns:
        distance: minimum distance between hyperplanes
    """
    v1_norm = v1 / np.linalg.norm(v1)
    v2_norm = v2 / np.linalg.norm(v2)
    dist1 = np.linalg.norm(v1_norm - v2_norm)
    dist2 = np.linalg.norm(v1_norm + v2_norm)
    return min(dist1, dist2)

def analyze_hyperplane_clustering(trained_model, w_list_odt, 
                                  distance_thresholds=[0.1, 0.2, 0.3]):
    """
    Analyze hyperplane clustering between trained DLGN and ODT.
    For each ODT hyperplane, counts how many DLGN hyperplanes are within
    each distance threshold.
    
    Args:
        trained_model: trained DLGN model
        w_list_odt: ODT hyperplanes (num_internal_nodes, input_dim)
        distance_thresholds: list of distance thresholds to check
    
    Returns:
        results: dict mapping threshold -> list of counts per ODT node
        closest_distances: closest distance for each ODT hyperplane
    """
    # Extract DLGN hyperplanes from all layers
    effective_weights = trained_model.return_gating_functions()
    dlgn_hyperplanes = []
    for layer_weights in effective_weights:
        for hyperplane in layer_weights:
            dlgn_hyperplanes.append(hyperplane.detach().numpy())
    
    num_internal_nodes = len(w_list_odt)
    results = {dist: [] for dist in distance_thresholds}
    closest_distances = []
    
    print("\nHyperplane Clustering Analysis")
    print("=" * 80)
    print(f"{'Distance':<12}", end="")
    for i in range(num_internal_nodes):
        print(f"Node{i:<2}", end=" ")
    print()
    print("-" * 80)

    # For each distance threshold, count nearby DLGN hyperplanes
    for dist_threshold in distance_thresholds:
        print(f"{dist_threshold:<12.2f}", end="")
        
        for odt_idx in range(num_internal_nodes):
            odt_hyperplane = w_list_odt[odt_idx]
            count = 0
            
            for dlgn_hyperplane in dlgn_hyperplanes:
                distance = compute_hyperplane_distance(odt_hyperplane, dlgn_hyperplane)
                if distance <= dist_threshold:
                    count += 1
            
            results[dist_threshold].append(count)
            print(f"{count:<5}", end=" ")
        print()

    # Compute closest distances
    print(f"{'Closest':<12}", end="")
    for odt_idx in range(num_internal_nodes):
        odt_hyperplane = w_list_odt[odt_idx]
        min_distance = float('inf')
        
        for dlgn_hyperplane in dlgn_hyperplanes:
            distance = compute_hyperplane_distance(odt_hyperplane, dlgn_hyperplane)
            min_distance = min(min_distance, distance)
        
        closest_distances.append(min_distance)
        print(f"{min_distance:<5.3f}", end=" ")
    print("\n" + "=" * 80)
    
    return results, closest_distances

# Data generation function
def data_gen_decision_tree(num_data=1000, dim=2, seed=0, w_list=None, b_list=None, 
                           vals=None, num_levels=3, threshold=1e-6, num_classes=4):        
    set_npseed(seed=seed)
    num_internal_nodes = 2**num_levels - 1
    num_leaf_nodes = 2**num_levels
    stats = np.zeros(num_internal_nodes+num_leaf_nodes)

    if vals is None:
        vals = np.arange(0, num_internal_nodes+num_leaf_nodes, 1, dtype=np.int32)
        vals[:num_internal_nodes] = -99
        
        # Assign classes to leaf nodes
        leaf_indices = np.arange(num_internal_nodes, num_internal_nodes+num_leaf_nodes)
        class_assignments = np.tile(np.arange(num_classes), (num_leaf_nodes // num_classes) + 1)[:num_leaf_nodes]
        np.random.shuffle(class_assignments)
        vals[leaf_indices] = class_assignments

    if w_list is None:
        w_list = np.random.randn(num_internal_nodes, dim)
        w_list /= np.linalg.norm(w_list, axis=1)[:, None]
        b_list = np.random.uniform(-0.5, 0.5, size=(num_internal_nodes,))

    oversample_factor = 2.0
    num_data_initial = int(num_data * oversample_factor)
    
    data_x = np.random.uniform(-1, 1, size=(num_data_initial, dim))
    relevant_stats = data_x @ w_list.T + b_list
    curr_index = np.zeros(shape=(num_data_initial), dtype=int)
    
    for level in range(num_levels):
        decision_variable = np.choose(curr_index, relevant_stats.T)
        curr_index = (curr_index+1)*2 - (1-(decision_variable > 0))
        
    bound_dist = np.min(np.abs(relevant_stats), axis=1)
    labels = vals[curr_index]
    
    data_x_pruned = data_x[bound_dist>threshold]
    labels_pruned = labels[bound_dist>threshold]
    
    samples_per_class = num_data // num_classes
    
    balanced_data = []
    balanced_labels = []
    
    for cls in range(num_classes):
        cls_mask = labels_pruned == cls
        cls_data = data_x_pruned[cls_mask]
        
        if len(cls_data) >= samples_per_class:
            indices = np.random.choice(len(cls_data), samples_per_class, replace=False)
            balanced_data.append(cls_data[indices])
            balanced_labels.append(np.full(samples_per_class, cls))
        else:
            indices = np.random.choice(len(cls_data), samples_per_class, replace=True)
            balanced_data.append(cls_data[indices])
            balanced_labels.append(np.full(samples_per_class, cls))
    
    data_x_balanced = np.vstack(balanced_data)
    labels_balanced = np.concatenate(balanced_labels)
    
    shuffle_idx = np.random.permutation(len(data_x_balanced))
    data_x_balanced = data_x_balanced[shuffle_idx]
    labels_balanced = labels_balanced[shuffle_idx]
    
    relevant_stats = np.sign(data_x_balanced @ w_list.T + b_list)
    nodes_active = np.zeros((len(data_x_balanced), num_internal_nodes+num_leaf_nodes), dtype=np.int32)
    for node in range(num_internal_nodes+num_leaf_nodes):
        if node==0:
            stats[node]=len(relevant_stats)
            nodes_active[:,0]=1
            continue
        parent = (node-1)//2
        nodes_active[:,node]=nodes_active[:,parent]
        right_child = node-(parent*2)-1
        if right_child==1:
            nodes_active[:,node] *= relevant_stats[:,parent]>0
        if right_child==0:
            nodes_active[:,node] *= relevant_stats[:,parent]<0
        stats = nodes_active.sum(axis=0)
    
    return ((data_x_balanced, labels_balanced), (w_list, b_list, vals), stats)

# DLGN Model (Multiclass)
class DLGN_FC(nn.Module):
    def __init__(self, input_dim=None, output_dim=None, num_hidden_nodes=[], 
                 beta=30, dlgn_mode='dlgn_sf', mode='pwc', num_classes=4):		
        super(DLGN_FC, self).__init__()
        self.num_hidden_layers = len(num_hidden_nodes)
        self.beta = beta  # Soft gating parameter
        self.dlgn_mode = dlgn_mode
        self.mode = mode
        self.num_classes = num_classes  # CHANGE: Added num_classes
        # CHANGE: output_dim should equal num_classes
        self.num_nodes = [input_dim] + num_hidden_nodes + [num_classes]
        self.gating_layers = nn.ModuleList()
        self.value_layers = nn.ModuleList()

        for i in range(self.num_hidden_layers+1):
            if i != self.num_hidden_layers:
                if self.dlgn_mode == 'dlgn_sf':
                    temp = nn.Linear(self.num_nodes[0], self.num_nodes[i+1], bias=False)
                else:
                    temp = nn.Linear(self.num_nodes[i], self.num_nodes[i+1], bias=False)
                self.gating_layers.append(temp)
            temp = nn.Linear(self.num_nodes[i], self.num_nodes[i+1], bias=False)
            self.value_layers.append(temp)
    
    def set_parameters_with_mask(self, to_copy, parameter_masks):
        for (name, copy_param) in to_copy.named_parameters():
            copy_param = copy_param.clone().detach()
            orig_param = self.state_dict()[name]
            if name in parameter_masks:
                param_mask = parameter_masks[name] > 0
                orig_param[param_mask] = copy_param[param_mask]
            else:
                orig_param = copy_param.data.detach()

    def return_gating_functions(self):
        effective_weights = []
        for i in range(self.num_hidden_layers):
            curr_weight = self.gating_layers[i].weight.detach().clone()
            if self.dlgn_mode == 'dlgn_sf':
                effective_weights.append(curr_weight)
            else:
                if i == 0:
                    effective_weights.append(curr_weight)
                else:
                    effective_weights.append(torch.matmul(curr_weight, effective_weights[-1]))
        return effective_weights

    def forward(self, x):
        gate_scores = [x]

        for el in self.parameters():
            if el.is_cuda:
                device = torch.device('cuda:0')
            else:
                device = torch.device('cpu')
        
        if self.mode == 'pwc':
            values = [torch.ones(x.shape).to(device)]
        else:
            values = [x.to(device)]

        for i in range(self.num_hidden_layers):
            if self.dlgn_mode == 'dlgn_sf':
                gate_scores.append((x @ self.gating_layers[i].weight.T))
            else:
                gate_scores.append(self.gating_layers[i].to(device)(gate_scores[-1].to(device)))
            curr_gate_on_off = torch.sigmoid(self.beta * gate_scores[-1])
            values.append(self.value_layers[i](values[-1]) * curr_gate_on_off)
        
        values.append(self.value_layers[self.num_hidden_layers](values[-1]))
        return values, gate_scores

# Training function (Multiclass)
def train_dlgn(DLGN_obj, train_data_curr, vali_data_curr, test_data_curr,
               train_labels_curr, test_labels_curr, vali_labels_curr, 
               num_epoch=1, parameter_mask=None, seed=42, lr=0.001, 
               no_of_batches=10, saved_epochs=None, x_epoch=1000):
    
    if parameter_mask is None:
        parameter_mask = {name: torch.ones_like(param) 
                         for name, param in DLGN_obj.named_parameters()}
    
    if saved_epochs is None:
        saved_epochs = list(range(0, num_epoch, 100)) + [num_epoch-1]
    
    print("train_data_curr inside train_dlgn:", train_data_curr.shape)
    set_torchseed(seed)

    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    DLGN_obj.to(device)
    
    # CHANGE: Use CrossEntropyLoss directly for multiclass
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(DLGN_obj.parameters(), lr=lr)

    train_data_torch = torch.Tensor(train_data_curr)
    vali_data_torch = torch.Tensor(vali_data_curr)
    test_data_torch = torch.Tensor(test_data_curr)

    train_labels_torch = torch.tensor(train_labels_curr, dtype=torch.int64)
    test_labels_torch = torch.tensor(test_labels_curr, dtype=torch.int64)
    vali_labels_torch = torch.tensor(vali_labels_curr, dtype=torch.int64)

    num_batches = no_of_batches
    batch_size = len(train_data_curr) // num_batches
    losses = []
    DLGN_obj_store = []
    best_vali_error = len(vali_labels_curr)
    debug_models = []
    train_losses = []

    for epoch in tqdm(range(saved_epochs[-1]+1)):
        if epoch in saved_epochs:
            DLGN_obj_copy = deepcopy(DLGN_obj)
            DLGN_obj_copy.to(torch.device('cpu'))
            DLGN_obj_store.append(DLGN_obj_copy)
            
            train_outputs_values, train_outputs_gate_scores = DLGN_obj(
                torch.Tensor(train_data_curr).to(device))
            train_preds = train_outputs_values[-1]
            
            # CHANGE: No need to concatenate, output is already [batch, num_classes]
            targets = torch.tensor(train_labels_curr, dtype=torch.int64).to(device)
            train_loss = criterion(train_preds, targets)

            train_losses.append(train_loss)
            if epoch % 1500 == 0 and epoch!=0:
                print(f"Epoch {epoch}, Train Loss: {train_loss.item():.6f}")
            if train_loss < 5e-6:
                break
            if np.isnan(train_loss.detach().cpu().numpy()):
                break
        
        running_loss = 0.0
        for batch_start in range(0, len(train_data_curr), batch_size):
            if (batch_start + batch_size) > len(train_data_curr):
                break
            
            optimizer.zero_grad()
            inputs = train_data_torch[batch_start:batch_start+batch_size]
            targets = train_labels_torch[batch_start:batch_start+batch_size].reshape(batch_size)
            inputs = inputs.to(device)
            targets = targets.to(device)
            
            values, gate_scores = DLGN_obj(inputs)
            # CHANGE: Output is already [batch, num_classes], no concatenation needed
            outputs = values[-1]
            loss = criterion(outputs, targets)
            loss.backward()
            
            for name, param in DLGN_obj.named_parameters():
                parameter_mask[name] = parameter_mask[name].to(device)
                param.grad *= parameter_mask[name]   
                if "gat" in name and epoch > x_epoch:
                    param.grad *= 0.
            
            optimizer.step()
            running_loss += loss.item()

        # Validation
        train_outputs_values, train_outputs_gate_scores = DLGN_obj(
            torch.Tensor(train_data_curr).to(device))
        train_preds = train_outputs_values[-1]
        targets = torch.tensor(train_labels_curr, dtype=torch.int64).to(device)
        train_loss = criterion(train_preds, targets)

        losses.append(train_loss.cpu().detach().clone().numpy())
        
        inputs = vali_data_torch.to(device)
        targets = vali_labels_torch.to(device)
        values, gate_scores = DLGN_obj(inputs)
        
        # CHANGE: Output is [batch, num_classes], take argmax directly
        vali_preds = torch.argmax(values[-1], dim=1)
        vali_error = torch.sum(targets != vali_preds)
        
        if vali_error < best_vali_error:
            DLGN_obj_return = deepcopy(DLGN_obj)
            best_vali_error = vali_error
            
    DLGN_obj_return.to(torch.device('cpu'))
    return train_losses, DLGN_obj_return, DLGN_obj_store, losses, debug_models



In [2]:
for num_data1 in [40000,60000,100000]:
#for beta1 in [4,5,6]:
    for num_classes1 in [3,4,5]:
        for input_dim1 in [30,40,50]:
            print("="*60)
            print("DLGN Multiclass Classification Training")
            print("="*60)
            
            # Configuration
            seed = 365
            num_levels = 4
            threshold = 0
            num_classes = num_classes1
            input_dim = input_dim1
            num_data = num_data1
            
            print(f"\nDataset Configuration:")
            print(f"  Input Dimension: {input_dim}")
            print(f"  Total Samples: {num_data}")
            print(f"  Number of Classes: {num_classes}")
            print(f"  Tree Levels: {num_levels}")
            
            # Generate data
            print("\nGenerating data...")
            ((data_x, labels), (w_list, b_list, vals), stats) = data_gen_decision_tree(
                dim=input_dim, 
                seed=seed, 
                num_levels=num_levels,
                num_data=num_data,
                num_classes=num_classes
            )
            device='cpu'
            print(f"\nLabel distribution:")
            for i in range(num_classes):
                print(f"  Class {i}: {sum(labels == i)}")
            
            # Split data
            num_train = num_data // 2
            num_vali = num_data // 4
            num_test = num_data // 4
            
            train_data = data_x[:num_train, :]
            train_data_labels = labels[:num_train]
            
            vali_data = data_x[num_train:num_train + num_vali, :]
            vali_data_labels = labels[num_train:num_train + num_vali]
            
            test_data = data_x[num_train + num_vali:, :]
            test_data_labels = labels[num_train + num_vali:]
            
            print(f"\nData split:")
            print(f"  Training: {len(train_data)}")
            print(f"  Validation: {len(vali_data)}")
            print(f"  Test: {len(test_data)}")
            
            # Model configuration
            dlgn_mode = 'dlgn'
            beta = 4#beta1
            lr = 0.002
            num_hidden_layers = 4
            num_hidden_nodes = [20, 20, 20, 20]
            no_of_batches = 10
            x_epoch = 1500
            saved_epochs = list(range(0, 1501, 100))
            
            print(f"\nModel Configuration:")
            print(f"  Mode: {dlgn_mode}")
            print(f"  Beta: {beta}")
            print(f"  Learning Rate: {lr}")
            print(f"  Hidden Layers: {num_hidden_layers}")
            print(f"  Hidden Nodes: {num_hidden_nodes}")
            
            # Initialize model
            set_torchseed(6675)
            DLGN_init = DLGN_FC(
                input_dim=input_dim, 
                output_dim=1,  # 1 output per class
                num_hidden_nodes=num_hidden_nodes, 
                beta=beta, 
                dlgn_mode=dlgn_mode,
                mode='pwc',
                num_classes=num_classes
            )
            
            # Setup parameter masks
            train_parameter_masks = dict()
            for name, parameter in DLGN_init.named_parameters():
                train_parameter_masks[name] = torch.ones_like(parameter).to(device)
            
            # Train model
            print("\n" + "="*60)
            print("Training...")
            print("="*60)
            
            set_torchseed(5000)
            train_losses, DLGN_obj_final, DLGN_obj_store, losses, debug_models = train_dlgn(
                train_data_curr=train_data,
                vali_data_curr=vali_data,
                test_data_curr=test_data,
                train_labels_curr=train_data_labels,
                vali_labels_curr=vali_data_labels,
                test_labels_curr=test_data_labels,
                DLGN_obj=deepcopy(DLGN_init),
                parameter_mask=train_parameter_masks,
                lr=lr,
                no_of_batches=no_of_batches,
                saved_epochs=saved_epochs,
                x_epoch=x_epoch
            )
            
            # Evaluate on test set
            print("\n" + "="*60)
            print("Evaluation")
            print("="*60)
            
            test_outputs_values, test_outputs_gate_scores = DLGN_obj_final(torch.Tensor(test_data))
            test_logits = test_outputs_values[-1].detach().numpy()
            test_preds = np.argmax(test_logits, axis=1)
            
            test_error = np.sum(test_data_labels != test_preds)
            num_test_data = len(test_data_labels)
            
            print(f"\nOverall Results:")
            print(f"  Test Accuracy: {1 - test_error / num_test_data:.5f}")
            print(f"  Correct: {num_test_data - test_error}/{num_test_data}")
            print(f"  Incorrect: {test_error}/{num_test_data}")
            
            # Per-class accuracy
            print(f"\nPer-Class Accuracy:")
            for i in range(num_classes):
                class_mask = test_data_labels == i
                class_correct = np.sum((test_data_labels[class_mask] == test_preds[class_mask]))
                class_total = np.sum(class_mask)
                if class_total > 0:
                    print(f"  Class {i}: {class_correct / class_total:.5f} ({class_correct}/{class_total})")
            
            print("\n" + "="*60)
            print("Training Complete!")
            print("="*60)
            # ============================================================================
            # 5. HYPERPLANE ANALYSIS
            # ============================================================================
            
            def compute_hyperplane_distance(v1, v2):
                """
                Compute distance between two hyperplanes.
                Uses minimum of ||v1_norm - v2_norm|| and ||v1_norm + v2_norm|| 
                to account for sign ambiguity.
                
                Args:
                    v1: first hyperplane normal vector
                    v2: second hyperplane normal vector
                
                Returns:
                    distance: minimum distance between hyperplanes
                """
                v1_norm = v1 / np.linalg.norm(v1)
                v2_norm = v2 / np.linalg.norm(v2)
                dist1 = np.linalg.norm(v1_norm - v2_norm)
                dist2 = np.linalg.norm(v1_norm + v2_norm)
                return min(dist1, dist2)
        
            def analyze_hyperplane_clustering(trained_model, w_list_odt, 
                                              distance_thresholds=[0.1, 0.2, 0.3]):
                """
                Analyze hyperplane clustering between trained DLGN and ODT.
                For each ODT hyperplane, counts how many DLGN hyperplanes are within
                each distance threshold.
                
                Args:
                    trained_model: trained DLGN model
                    w_list_odt: ODT hyperplanes (num_internal_nodes, input_dim)
                    distance_thresholds: list of distance thresholds to check
                
                Returns:
                    results: dict mapping threshold -> list of counts per ODT node
                    closest_distances: closest distance for each ODT hyperplane
                """
                # Extract DLGN hyperplanes from all layers and classes
                effective_weights = trained_model.return_gating_functions()
                dlgn_hyperplanes = []
                for layer_weights in effective_weights:
                    for hyperplane in layer_weights:
                        dlgn_hyperplanes.append(hyperplane.detach().numpy())
                
                num_internal_nodes = len(w_list_odt)
                results = {dist: [] for dist in distance_thresholds}
                closest_distances = []
                
                print("\nHyperplane Clustering Analysis")
                print("=" * 80)
                print(f"{'Distance':<12}", end="")
                for i in range(num_internal_nodes):
                    print(f"Node{i:<2}", end=" ")
                print()
                print("-" * 80)
            
                # For each distance threshold, count nearby DLGN hyperplanes
                for dist_threshold in distance_thresholds:
                    print(f"{dist_threshold:<12.2f}", end="")
                    
                    for odt_idx in range(num_internal_nodes):
                        odt_hyperplane = w_list_odt[odt_idx]
                        count = 0
                        
                        for dlgn_hyperplane in dlgn_hyperplanes:
                            distance = compute_hyperplane_distance(odt_hyperplane, dlgn_hyperplane)
                            if distance <= dist_threshold:
                                count += 1
                        
                        results[dist_threshold].append(count)
                        print(f"{count:<5}", end=" ")
                    print()
            
                # Compute closest distances
                print(f"{'Closest':<12}", end="")
                for odt_idx in range(num_internal_nodes):
                    odt_hyperplane = w_list_odt[odt_idx]
                    min_distance = float('inf')
                    
                    for dlgn_hyperplane in dlgn_hyperplanes:
                        distance = compute_hyperplane_distance(odt_hyperplane, dlgn_hyperplane)
                        min_distance = min(min_distance, distance)
                    
                    closest_distances.append(min_distance)
                    print(f"{min_distance:<5.3f}", end=" ")
                print("\n" + "=" * 80)
            
            
                return results, closest_distances
        
        
            # ============================================================================
            # Usage example (add after training completes)
            # ============================================================================
            
            # Analyze hyperplane clustering
            print("\n" + "="*60)
            print("Hyperplane Clustering Analysis")
            print("="*60)
            
            clustering_results, closest_dists = analyze_hyperplane_clustering(
                trained_model=DLGN_obj_final,
                w_list_odt=w_list,
                distance_thresholds=[0.1, 0.2, 0.3]
            )
            

DLGN Multiclass Classification Training

Dataset Configuration:
  Input Dimension: 30
  Total Samples: 40000
  Number of Classes: 3
  Tree Levels: 4

Generating data...

Label distribution:
  Class 0: 13333
  Class 1: 13333
  Class 2: 13333

Data split:
  Training: 20000
  Validation: 10000
  Test: 9999

Model Configuration:
  Mode: dlgn
  Beta: 4
  Learning Rate: 0.002
  Hidden Layers: 4
  Hidden Nodes: [20, 20, 20, 20]

Training...
train_data_curr inside train_dlgn: (20000, 30)


100%|██████████| 1501/1501 [01:50<00:00, 13.60it/s]


Epoch 1500, Train Loss: 0.031172

Evaluation

Overall Results:
  Test Accuracy: 0.90339
  Correct: 9033/9999
  Incorrect: 966/9999

Per-Class Accuracy:
  Class 0: 0.92757 (3099/3341)
  Class 1: 0.90186 (2950/3271)
  Class 2: 0.88102 (2984/3387)

Training Complete!

Hyperplane Clustering Analysis

Hyperplane Clustering Analysis
Distance    Node0  Node1  Node2  Node3  Node4  Node5  Node6  Node7  Node8  Node9  Node10 Node11 Node12 Node13 Node14 
--------------------------------------------------------------------------------
0.10        0     0     1     0     0     0     0     0     0     0     0     0     0     0     0     
0.20        2     0     4     0     0     1     0     0     0     0     0     0     0     0     0     
0.30        2     0     5     0     0     3     0     0     0     0     0     1     0     1     0     
Closest     0.159 0.405 0.069 0.340 1.014 0.169 0.844 0.556 1.103 0.925 0.968 0.294 0.546 0.216 0.717 
DLGN Multiclass Classification Training

Dataset Configurati

100%|██████████| 1501/1501 [01:53<00:00, 13.21it/s]


Epoch 1500, Train Loss: 0.076553

Evaluation

Overall Results:
  Test Accuracy: 0.88719
  Correct: 8871/9999
  Incorrect: 1128/9999

Per-Class Accuracy:
  Class 0: 0.91077 (3052/3351)
  Class 1: 0.85279 (2856/3349)
  Class 2: 0.89815 (2963/3299)

Training Complete!

Hyperplane Clustering Analysis

Hyperplane Clustering Analysis
Distance    Node0  Node1  Node2  Node3  Node4  Node5  Node6  Node7  Node8  Node9  Node10 Node11 Node12 Node13 Node14 
--------------------------------------------------------------------------------
0.10        0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     
0.20        4     0     4     0     0     0     0     0     0     0     1     1     0     0     0     
0.30        9     0     4     0     0     0     0     0     0     0     1     1     0     0     0     
Closest     0.108 0.398 0.104 1.082 0.956 0.346 0.486 1.033 1.082 1.052 0.111 0.135 0.628 0.743 0.455 
DLGN Multiclass Classification Training

Dataset Configurat

...

Epoch 1500, Train Loss: 0.080675


100%|██████████| 1501/1501 [04:20<00:00,  5.77it/s]



Evaluation

Overall Results:
  Test Accuracy: 0.94696
  Correct: 23674/25000
  Incorrect: 1326/25000

Per-Class Accuracy:
  Class 0: 0.95787 (4774/4984)
  Class 1: 0.95311 (4838/5076)
  Class 2: 0.93456 (4727/5058)
  Class 3: 0.94144 (4614/4901)
  Class 4: 0.94780 (4721/4981)

Training Complete!

Hyperplane Clustering Analysis

Hyperplane Clustering Analysis
Distance    Node0  Node1  Node2  Node3  Node4  Node5  Node6  Node7  Node8  Node9  Node10 Node11 Node12 Node13 Node14 
--------------------------------------------------------------------------------
0.10        9     0     3     0     0     0     1     0     0     0     0     0     0     0     0     
0.20        12    3     5     0     0     0     1     0     1     0     1     1     0     0     1     
0.30        14    5     6     0     1     1     2     0     1     0     1     1     0     0     3     
Closest     0.046 0.131 0.032 0.438 0.228 0.243 0.047 0.791 0.118 0.899 0.157 0.198 1.230 1.058 0.127 
DLGN Multiclass Classificat

100%|██████████| 1501/1501 [03:23<00:00,  7.37it/s]


Epoch 1500, Train Loss: 0.091058

Evaluation

Overall Results:
  Test Accuracy: 0.93900
  Correct: 23475/25000
  Incorrect: 1525/25000

Per-Class Accuracy:
  Class 0: 0.91987 (4569/4967)
  Class 1: 0.95611 (4771/4990)
  Class 2: 0.93554 (4644/4964)
  Class 3: 0.95121 (4718/4960)
  Class 4: 0.93241 (4773/5119)

Training Complete!

Hyperplane Clustering Analysis

Hyperplane Clustering Analysis
Distance    Node0  Node1  Node2  Node3  Node4  Node5  Node6  Node7  Node8  Node9  Node10 Node11 Node12 Node13 Node14 
--------------------------------------------------------------------------------
0.10        0     3     0     2     1     0     1     1     2     0     0     0     0     0     1     
0.20        5     8     1     2     1     0     1     1     2     1     1     0     0     0     1     
0.30        6     9     2     4     1     0     1     1     2     1     1     0     0     0     1     
Closest     0.135 0.057 0.176 0.028 0.080 1.144 0.067 0.096 0.025 0.143 0.107 0.373 1.237 1.156 0